# Импорт необходимых библиотек

In [ ]:
import numpy as np
import pandas as pd
import datetime
import random

import optuna
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, LSTM, Dense, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from keras.models import save_model

# Создание функций

## Создание мок-данных с датами

In [ ]:
def generate_dates_for_client(sub_df, start_date):
    category_to_date = {}  # Словарь для хранения даты для определенной категории

    for index, row in sub_df.iterrows():
        category = row['category']

        # Если категория уже имеет дату и случайное число меньше 0.5 (50% шанс), то используем эту дату
        if category in category_to_date and np.random.rand() < 0.5:
            new_date = category_to_date[category]
        else:
            # В противном случае генерируем новую дату
            delta = datetime.timedelta(
                days=np.random.randint(0, 365*2),
                seconds=np.random.randint(0, 3600*24),
            )
            new_date = start_date + delta
            category_to_date[category] = new_date  # Сохраняем дату для данной категории

        sub_df.loc[index, 'date'] = new_date

    return sub_df

## Создание временных фичей

In [ ]:
def add_time_features(df):
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['season'] = (df['month'] % 12 + 3) // 3 # 1: зима, 2: весна, 3: лето, 4: осень
    return df

## Создание датасетов для обучения модели

In [ ]:
def create_dataset(data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(data) - look_back):
        dataX.append(data[i:(i + look_back), :])
        dataY.append(data[i + look_back, :])
    return np.array(dataX), np.array(dataY)

## Создание нейросети

In [ ]:
def lstm(trial):
    look_back = trial.suggest_int('look_back', 1, 23)

    # Выделение последнего месяца как тестового набора данных
    test = df_grouped.iloc[-1].values.reshape(1, -1)
    train = df_grouped.iloc[:-1]

    # Масштабирование
    scaler = StandardScaler().fit(train.values)
    scaled_train = scaler.transform(train.values)
    scaled_test = scaler.transform(test)

    X_train, y_train = create_dataset(scaled_train, look_back)
    X_test = scaled_train[-look_back:].reshape(1, look_back, -1)
    y_test = scaled_test

    n_neurons = trial.suggest_int('n_neurons', 4, 128)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    num_layers = trial.suggest_int('num_layers', 1, 10)
    dropout_rate = trial.suggest_float('dropout_rate', 0, 0.5)
    epochs = trial.suggest_int('epochs', 5, 100)
    batch_size = trial.suggest_int('batch_size', 1, 256)

    model = build_and_train_lstm(X_train, y_train, n_neurons, Adam, learning_rate, num_layers, dropout_rate, epochs, batch_size)
    y_pred = model.predict(X_test)

    # Денормализация прогноза
    y_pred = scaler.inverse_transform(y_pred)

    trial.set_user_attr("scaler", scaler)

    return mean_squared_error(test, y_pred)


## Создание и обучение модели

In [ ]:
def build_and_train_lstm(X_train, y_train, n_neurons, optimizer, learning_rate, num_layers, dropout_rate, epochs, batch_size):

    input_time_series = Input(shape=(X_train.shape[1], X_train.shape[2]), name='time_series_input')

    x_ts = input_time_series
    for i in range(num_layers):
        x_ts = LSTM(n_neurons, return_sequences=True if i < num_layers - 1 else False)(x_ts)
        if dropout_rate > 0:
            x_ts = Dropout(dropout_rate)(x_ts)

    output = Dense(y_train.shape[1])(x_ts)

    model = Model(inputs=input_time_series, outputs=output)

    model.summary()

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)  # change verbose to 1 to see training logs

    return model

# Обработка данных

Словарь с шифром

In [ ]:
dictionary = {
    "topic": ['автозапчасти', 'видеоигры', 'напитки', 'продукты питания', 'закуски и приправы', 'аквариум', 'одежда', 'уборка', 'электроника', 'образование'],
    "label": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

## Загружаем датасет и обрабатываем данные

In [ ]:
df = pd.read_excel('final_df.xlsx', sheet_name='Sheet1')
df = df[df['topic']!='нет категории']

Создание мок-данных с датами

In [ ]:
unique_clients = df['client'].unique()
selected_clients = np.random.choice(unique_clients, 2400, replace=False)

# Создаем список для сохранения промежуточных результатов
all_subsets = []

for i in range(100):
    # Выберите 24 клиента для текущей группы
    clients_group = selected_clients[i*24:(i+1)*24]

    client_name = "Объединенный_Клиент_" + str(i+1)
    start_date = datetime.datetime.now() - datetime.timedelta(days=365*2)  # начнем отсчет с 2-х лет назад

    for client in clients_group:
        client_subset = df[df['client'] == client].copy()
        client_subset['date'] = None
        client_subset = generate_dates_for_client(client_subset, start_date)
        client_subset['client'] = client_name
        all_subsets.append(client_subset)

# Используем pd.concat для объединения всех подмножеств
result_df = pd.concat(all_subsets, ignore_index=True)

In [ ]:
display(result_df[result_df['client']=='Объединенный_Клиент_2'].sort_values(by=['date']).tail(60))

,Столбец1,sale,category,price,client,cleaned_sale,topic,Unnamed: 7,Категория,Встречаемость,Unnamed: 10,Unnamed: 11,Unnamed: 12,date
1135,259938,Напиток чайный травяной о'кей русский сбор...,1976872004,74,Объединенный_Клиент_2,напиток чайный травяной русский сбор,напитки,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-23 22:26:53.350929
1406,295362,Вода природная питьевая сенежская газированная...,1976872004,38,Объединенный_Клиент_2,вода природный питьевой сенежский газированная л,напитки,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 03:03:58.350929
1407,295363,Кофе carte noire intense absolu в зернах 800 г,1976872004,2,Объединенный_Клиент_2,кофе carte noire intense absolu в зернах г,напитки,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 03:03:58.350929
799,206596,Оперативная память kingston fury beast black [...,1838685550,3899,Объединенный_Клиент_2,оперативный память kingston fury beast black г...,электроника,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 05:16:08.350929
800,206597,Материнская плата asrock b660m pro rs,1838685550,8466,Объединенный_Клиент_2,материнский плата asrock m pro rs,электроника,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-25 05:16:08.350929
1328,69197,Конфеты шоколадные essen 35 с морской солью и ...,1363498391,64,Объединенный_Клиент_2,конфета шоколадный essen с морской соль и кран...,продукты питания,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-27 05:50:46.350929
1327,69196,Вафли акульчев венские с бананом 100 г,1363498391,59,Объединенный_Клиент_2,вафли акульчев венский с банан г,продукты питания,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-27 05:50:46.350929
1056,158669,Мешок для стирки одежды atmosphere 40 х 30 см,1505938033,339,Объединенный_Клиент_2,мешок для стирка одежда atmosphere х см,уборка,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-31 17:15:19.350929
1057,158670,Сменный флакон средства mr. muscle утренняя ро...,1505938033,189,Объединенный_Клиент_2,сменный флакон средство mr muscle утренний рос...,уборка,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-31 17:15:19.350929
802,247922,Смесь орехов и кукурузы ем! best nuts super ba...,1964648565,229,Объединенный_Клиент_2,смесь орех и кукуруза ем best nuts super bar г,закуски и приправы,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-02 01:32:45.350929


Даты создались корректно

Преобразование типа данных

In [ ]:
result_df['date'] = pd.to_datetime(result_df['date'])

Сохраняем одного клиента для будущих тестов

In [ ]:
test_df = result_df[result_df['client']=='Объединенный_Клиент_1']

In [ ]:
test_df.to_csv('test_df.csv')

In [ ]:
train_df = result_df.drop(result_df[result_df.index.isin(test_df.index)].index)

Добавление временных фичей в трейн и тест выборки

In [ ]:
new_train_df = add_time_features(train_df)
new_test_df = add_time_features(test_df)

C:\Users\User\AppData\Local\Temp\ipykernel_21556\553976233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'] = df['date'].dt.quarter
C:\Users\User\AppData\Local\Temp\ipykernel_21556\553976233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['date'].dt.month
C:\Users\User\AppData\Local\Temp\ipykernel_21556\553976233.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

## Аггрегация данных

In [ ]:
df_grouped = new_train_df.groupby(['client', 'year', 'month', 'season', 'topic']).agg({'price': 'sum'}).reset_index()
df_grouped = df_grouped.pivot_table(index=['year', 'month', 'season'], columns='topic', values='price', fill_value=0).reset_index()

In [ ]:
df_grouped_test = new_test_df.groupby(['client', 'year', 'month', 'season', 'topic']).agg({'price': 'sum'}).reset_index()
df_grouped_test = df_grouped_test.pivot_table(index=['year', 'month', 'season'], columns='topic', values='price', fill_value=0).reset_index()

#  Запуск обучения

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(lstm, n_trials=20)

best_params = study.best_params
print("Лучшие параметры: ", best_params)

[I 2023-09-27 22:39:58,701] A new study created in memory with name: no-name-661fe10b-ab08-4037-b42f-f68e6147d57d


Model: "model_299"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_series_input (InputLa  [(None, 10, 10)]          0         
 yer)                                                            
                                                                 
 lstm_1474 (LSTM)            (None, 10, 77)            27104     
                                                                 
 dropout_1474 (Dropout)      (None, 10, 77)            0         
                                                                 
 lstm_1475 (LSTM)            (None, 10, 77)            47740     
                                                                 
 dropout_1475 (Dropout)      (None, 10, 77)            0         
                                                                 
 lstm_1476 (LSTM)            (None, 10, 77)            47740     
                                                         

[I 2023-09-27 22:40:23,480] Trial 0 finished with value: 14872399.034937257 and parameters: {'look_back': 10, 'n_neurons': 77, 'learning_rate': 0.03203437321567369, 'num_layers': 9, 'dropout_rate': 0.3589718194965184, 'epochs': 100, 'batch_size': 207}. Best is trial 0 with value: 14872399.034937257.


Model: "model_300"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_series_input (InputLa  [(None, 12, 10)]          0         
 yer)                                                            
                                                                 
 lstm_1483 (LSTM)            (None, 12, 97)            41904     
                                                                 
 dropout_1483 (Dropout)      (None, 12, 97)            0         
                                                                 
 lstm_1484 (LSTM)            (None, 12, 97)            75660     
                                                                 
 dropout_1484 (Dropout)      (None, 12, 97)            0         
                                                                 
 lstm_1485 (LSTM)            (None, 12, 97)            75660     
                                                         

[I 2023-09-27 22:40:33,424] Trial 1 finished with value: 9053509.538612772 and parameters: {'look_back': 12, 'n_neurons': 97, 'learning_rate': 0.002591718149280472, 'num_layers': 5, 'dropout_rate': 0.34731515204354824, 'epochs': 13, 'batch_size': 131}. Best is trial 1 with value: 9053509.538612772.


...

Model: "model_317"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_series_input (InputLa  [(None, 19, 10)]          0         
 yer)                                                            
                                                                 
 lstm_1570 (LSTM)            (None, 19, 24)            3360      
                                                                 
 dropout_1570 (Dropout)      (None, 19, 24)            0         
                                                                 
 lstm_1571 (LSTM)            (None, 19, 24)            4704      
                                                                 
 dropout_1571 (Dropout)      (None, 19, 24)            0         
                                                                 
 lstm_1572 (LSTM)            (None, 19, 24)            4704      
                                                         

[I 2023-09-27 22:44:17,755] Trial 18 finished with value: 16795740.592632275 and parameters: {'look_back': 19, 'n_neurons': 24, 'learning_rate': 0.03693577908914035, 'num_layers': 10, 'dropout_rate': 0.2966031847991124, 'epochs': 42, 'batch_size': 51}. Best is trial 15 with value: 4735971.344863197.


Model: "model_318"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_series_input (InputLa  [(None, 13, 10)]          0         
 yer)                                                            
                                                                 
 lstm_1580 (LSTM)            (None, 13, 4)             240       
                                                                 
 dropout_1580 (Dropout)      (None, 13, 4)             0         
                                                                 
 lstm_1581 (LSTM)            (None, 13, 4)             144       
                                                                 
 dropout_1581 (Dropout)      (None, 13, 4)             0         
                                                                 
 lstm_1582 (LSTM)            (None, 13, 4)             144       
                                                         

[I 2023-09-27 22:44:30,620] Trial 19 finished with value: 8623958.311775438 and parameters: {'look_back': 13, 'n_neurons': 4, 'learning_rate': 0.03861256844675997, 'num_layers': 6, 'dropout_rate': 0.05804846707841893, 'epochs': 66, 'batch_size': 164}. Best is trial 15 with value: 4735971.344863197.


Лучшие параметры:  {'look_back': 22, 'n_neurons': 4, 'learning_rate': 0.03760326378601691, 'num_layers': 10, 'dropout_rate': 0.1638169303427387, 'epochs': 51, 'batch_size': 38}


Сохранение лучших параметров

In [ ]:
best_params = study.best_params
best_trial = study.best_trial
best_scaler = best_trial.user_attrs["scaler"]
best_look_back = best_trial.params['look_back']

In [ ]:
df_grouped_test.iloc[-1]

topic
автозапчасти           863
аквариум                 0
видеоигры                0
закуски и приправы      49
напитки               1805
образование           2040
одежда                   0
продукты питания      1971
уборка                  89
электроника              0
Name: 24, dtype: int64

## Проверка на тесте

In [ ]:
final_test = df_grouped_test.iloc[-1].values.reshape(1, -1)
final_train = df_grouped_test.iloc[:-1]

# Масштабирование
scaler = StandardScaler().fit(final_train.values)
final_scaled_train = scaler.transform(final_train.values)
final_scaled_test = scaler.transform(final_test)

X_train, y_train = create_dataset(final_scaled_train, best_look_back)
X_test = final_scaled_train[-best_look_back:].reshape(1, best_look_back, -1)
y_test = final_scaled_test

# Построение и обучение модели с лучшими параметрами на всем тренировочном наборе данных
best_model = build_and_train_lstm(
    X_train, y_train,
    n_neurons=best_params['n_neurons'],
    optimizer=Adam,
    learning_rate=best_params['learning_rate'],
    num_layers=best_params['num_layers'],
    dropout_rate=best_params['dropout_rate'],
    epochs=best_params['epochs'],
    batch_size=best_params['batch_size']
)

# 4. Тестирование модели на тестовом наборе данных
predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error on Test Data: {mse}")

Model: "model_319"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_series_input (InputLa  [(None, 22, 10)]          0         
 yer)                                                            
                                                                 
 lstm_1586 (LSTM)            (None, 22, 4)             240       
                                                                 
 dropout_1586 (Dropout)      (None, 22, 4)             0         
                                                                 
 lstm_1587 (LSTM)            (None, 22, 4)             144       
                                                                 
 dropout_1587 (Dropout)      (None, 22, 4)             0         
                                                                 
 lstm_1588 (LSTM)            (None, 22, 4)             144       
                                                         

## Дешифровка данных

In [ ]:
predictions_original = scaler.inverse_transform(predictions)

In [ ]:
print("\nСравнение реальных данных и прогноза:")

topics = ['автозапчасти', 'аквариум', 'видеоигры', 'закуски и приправы', 'напитки', 'образование', 'одежда', 'продукты питания', 'уборка', 'электроника']

for j, topic in enumerate(topics):
    print(f"{topic}: Реальные данные: {final_test[0][j]:.2f}, Прогноз: {predictions_original[0][j]:.2f}")


Сравнение реальных данных и прогноза:
автозапчасти: Реальные данные: 863.00, Прогноз: -34.07
аквариум: Реальные данные: 0.00, Прогноз: 1911.49
видеоигры: Реальные данные: 0.00, Прогноз: 233.48
закуски и приправы: Реальные данные: 49.00, Прогноз: 808.51
напитки: Реальные данные: 1805.00, Прогноз: 1004.00
образование: Реальные данные: 2040.00, Прогноз: 1225.51
одежда: Реальные данные: 0.00, Прогноз: 348.28
продукты питания: Реальные данные: 1971.00, Прогноз: 1509.28
уборка: Реальные данные: 89.00, Прогноз: 731.31
электроника: Реальные данные: 0.00, Прогноз: 55939.65


В целом результаты расходятся ощутимо в некоторых позициям, но это результат создания искусственных данных из разных людей. По сути модель предсказывает траты в следюущем месяце для обобщённого пользователя из двух десятков клиентов. А сравнение мы проводим с другим человеком. Однако сами числа в целом нормальные. Есть отрицательные значения, но это даже лучше, потому что мы не будем выводить эти числа пользователю, а ранжировать по ним категории. В этом случае отрицательное значение будет стоять меньше нуля и почти не имеет шансов попасть в пятёрку.

# Сохранение модели и параметров

In [ ]:
best_model.save("spendings.h5")

In [ ]:
best_look_back

22